In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoad().get_contract(web3, abi_path)
sync_filt = Filter.create_filter(address=None, event_types=[Pair.events.Sync])  # Listen events from any smart contract

In [4]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 10

In [6]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=sync_filt)

In [7]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])
    pair: PairDetails = FetchPairDetails().apply(web3, evt['address'])

    amt0 = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    amt1 = Conversion().convert_int256_bytes_to_int(arguments[1])  
    amt0_human = amt0/(10**pair.token0.decimals)
    amt1_human = amt1/(10**pair.token1.decimals)    

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt['event']
    event['details']['token0'] = pair.token0.address
    event['details']['token1'] = pair.token1.address
    event['details']['token0_symbol'] = pair.token0.symbol
    event['details']['token1_symbol'] = pair.token1.symbol 
    event['details']['token0_decimal'] = pair.token0.decimals
    event['details']['token1_decimal'] = pair.token1.decimals        
    event['details']['amount0'] = amt0
    event['details']['amount1'] = amt1
    event['details']['price'] = amt1_human/amt0_human
    
    events[k] = event
    if key not in processed_events:
        print(f"Sync at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Sync at block:65,321,838 tx:0x5e4d444f98cbfbe2ae71f0b53c3420a2024b7c99f37c6f777865a900aeb69c7c
Sync at block:65,321,838 tx:0x437094ba736532a7da76492ba7ba844b30c87ac450e48490e781b3cb3b5ddbf2
Sync at block:65,321,838 tx:0x813920ce87b043de08e98376aaef3c7862dedfaddb820d6a305e3be666033bf5
Sync at block:65,321,838 tx:0xfcaeb27655b0ae1fa12902ecfccb94ab72243c3b659ccb103bf73c7d9ac54d33
Sync at block:65,321,839 tx:0x8935647e125ccd007713ca6b04db16e05d0d08878a6be61a541f3a8b1e051462
Sync at block:65,321,839 tx:0xb6975018dd21f5800d5f1ced26f16b3cdb6cbda9149b387f29557457188a09fa
Sync at block:65,321,839 tx:0xeb4ecedd6415ff26de701789735acbef5102ca8a9de9f754457756b7b4dc4cf7
Sync at block:65,321,839 tx:0x7dd108ad5bcdac88e63da396b5ed7d7386b8da53288a388a36c63deffd1a90ae
Sync at block:65,321,839 tx:0x7dd108ad5bcdac88e63da396b5ed7d7386b8da53288a388a36c63deffd1a90ae
Sync at block:65,321,839 tx:0xbd407dfb316c111ba3e5a2cd2b30d90b67c100707a6ba6e49e76183cf27b5344
Sync at block:65,321,839 tx:0x632b8cc72f1df2f23bb5

In [8]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,sync,sushi,0x0c787944946d22922c9f41c477cc539700d35bb2,0x5e4d444f98cbfbe2ae71f0b53c3420a2024b7c99f37c...,65321838,1733855986,{'web3_type': <class 'web3._utils.datatypes.Sy...
1,polygon,uniswapv2pair,sync,sushi,0xa964b92c690f0b8efad5b46a278a84bd0ef8ac01,0x437094ba736532a7da76492ba7ba844b30c87ac450e4...,65321838,1733855986,{'web3_type': <class 'web3._utils.datatypes.Sy...
2,polygon,uniswapv2pair,sync,sushi,0xf87b839077db0e5ea5bf8b6460c8413abe72bc2b,0x813920ce87b043de08e98376aaef3c7862dedfaddb82...,65321838,1733855986,{'web3_type': <class 'web3._utils.datatypes.Sy...
3,polygon,uniswapv2pair,sync,sushi,0xc4e595acdd7d12fec385e5da5d43160e8a0bac0e,0xfcaeb27655b0ae1fa12902ecfccb94ab72243c3b659c...,65321838,1733855986,{'web3_type': <class 'web3._utils.datatypes.Sy...
4,polygon,uniswapv2pair,sync,sushi,0x2813d43463c374a680f235c428fb1d7f08de0b69,0x8935647e125ccd007713ca6b04db16e05d0d08878a6b...,65321839,1733855988,{'web3_type': <class 'web3._utils.datatypes.Sy...
...,...,...,...,...,...,...,...,...,...
81,polygon,uniswapv2pair,sync,sushi,0x58503919c6fec346c0379f8370467a2c29c09eb1,0xf6851c477ba462532846f3e7a3153e7ccf36a2818ccc...,65321847,1733856006,{'web3_type': <class 'web3._utils.datatypes.Sy...
82,polygon,uniswapv2pair,sync,sushi,0x4485df5f2b6bf03935cee44bf41ba65c06650b84,0x3f0d8e4fb7e1f694131a5d3323782adbe7be4e209dd3...,65321847,1733856006,{'web3_type': <class 'web3._utils.datatypes.Sy...
83,polygon,uniswapv2pair,sync,sushi,0x6cf8654e85ab489ca7e70189046d507eba233613,0x3f0d8e4fb7e1f694131a5d3323782adbe7be4e209dd3...,65321847,1733856006,{'web3_type': <class 'web3._utils.datatypes.Sy...
84,polygon,uniswapv2pair,sync,sushi,0xff2bbcb399ad50bbd06debadd47d290933ae1038,0xcbc7a149169ab163ad29796ce17426fd2325dc31e273...,65321847,1733856006,{'web3_type': <class 'web3._utils.datatypes.Sy...


In [9]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'sync',
  'platform': 'sushi',
  'address': '0x0c787944946d22922c9f41c477cc539700d35bb2',
  'tx_hash': '0x5e4d444f98cbfbe2ae71f0b53c3420a2024b7c99f37c6f777865a900aeb69c7c',
  'blk_num': 65321838,
  'timestamp': 1733855986,
  'details': {'web3_type': web3._utils.datatypes.Sync,
   'token0': '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619',
   'token1': '0xD6DF932A45C0f255f85145f286eA0b292B21C90B',
   'token0_symbol': 'WETH',
   'token1_symbol': 'AAVE',
   'token0_decimal': 18,
   'token1_decimal': 18,
   'amount0': 10818475097610843370,
   'amount1': 148372332483558583339,
   'price': 13.714717753181795}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'sync',
  'platform': 'sushi',
  'address': '0xa964b92c690f0b8efad5b46a278a84bd0ef8ac01',
  'tx_hash': '0x437094ba736532a7da76492ba7ba844b30c87ac450e48490e781b3cb3b5ddbf2',
  'blk_num': 65321838,
  'timestamp': 1733855986,
  'details': {'web3_type': web3._uti